In [53]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

In [54]:
%qtconsole

Calibration

In [73]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((4*4,3), np.float32)
objp[:,:2] = np.mgrid[0:4,0:4].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('data_calibration/cam_g/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (4,4),None)
    # If found, add object points, image points (after refining them)
    
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (4,4), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(1000)

cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9


In [56]:
#ret is how far are the projected points from the real points in px in average
#K is the intrinsic matrix
#dist is the vector of distorsion coefficients
#rvecs are the rotation vectors (one vector for one image)
#tvecs are the translation vectors
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

Erreur de reprojection

In [52]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print("total error: ", mean_error/len(objpoints))

total error:  0.12705796499976332


Rectification epipolaire 

In [125]:
T = np.array([[0.1],[0],[0]],dtype = "float64")
R = np.array([[1,0,0],[0,1,0],[0,0,1]], dtype = "float64")
dist = np.transpose(np.array(dist, dtype = "float64"))
K = np.array(K, dtype = "float64")

In [126]:
res = cv2.stereoRectify(K, dist, K, dist, (480,640), R,T)

In [130]:
for e in res:
    print(e)
    print('---------------------')

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
---------------------
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
---------------------
[[244.           0.         240.09179688   0.        ]
 [  0.         244.         319.99905291   0.        ]
 [  0.           0.           1.           0.        ]]
---------------------
[[244.           0.         240.09179688  24.4       ]
 [  0.         244.         319.99905291   0.        ]
 [  0.           0.           1.           0.        ]]
---------------------
[[   1.            0.            0.         -240.09179688]
 [   0.            1.            0.         -319.99905291]
 [   0.            0.            0.          244.        ]
 [   0.            0.          -10.            0.        ]]
---------------------
(0, 0, 0, 0)
---------------------
(0, 0, 0, 0)
---------------------
